In [1]:
import tensornetwork as tn
import numpy as np

In [33]:
def decimal_to_binary(num: int) -> str:
    return str(bin(num))[2:]

def pad_binary(binary_str: str, length: int) -> str:
    return '0'*(length-len(binary_str)) + binary_str

In [37]:
n = 4
N = 2**n
for i in range(N):
    print(pad_binary(decimal_to_binary(i), n))

0000
0001
0010
0011


In [17]:
2 / 2**1

1.0

In [5]:
a = np.array([1.0 + 0.0j, 0.0 + 0.0j])
b = np.array([0.0 + 0.0j, 1.0 + 0.0j])

h = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
I = np.array([[1, 0], [0, 1]])

a @ h @ h

array([1.+0.j, 0.+0.j])

In [10]:
u = np.kron(I, h)
c = np.kron(a, b)


u@c

array([ 0.70710678+0.j, -0.70710678+0.j,  0.        +0.j,  0.        +0.j])

In [12]:
u

array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.70710678, -0.70710678,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        , -0.        ,  0.70710678, -0.70710678]])

In [78]:
state = tn.Node(np.array([1.0 + 0.0j, 0.0 + 0.0j]))
qubit_in = state[0]

# This node represents the Hadamard gate we wish to perform
# on this qubit.
hadamard = tn.Node(np.array([[1, 1], [1, -1]])) / np.sqrt(2)
tn.connect(qubit_in, hadamard[0]) # Equal to qubit ^ hadamard[0]
# The "output edge" of the operation represents the qubit after
#  applying the operation.
qubit_out = hadamard[1]
# Contraction is how you actually "apply" the gate.
state2 = state @ hadamard
print(state2.tensor) # array([0.707+0.j, 0.707+0.j])

[0.70710678+0.j 0.70710678+0.j]


In [79]:
def apply_gate(qubit_edges, gate, operating_qubits):
  op = tn.Node(gate)
  for i, bit in enumerate(operating_qubits):
    tn.connect(qubit_edges[bit], op[i])
    qubit_edges[bit] = op[i + len(operating_qubits)]

# These are just numpy arrays of the operators.
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)
CNOT = np.zeros((2, 2, 2, 2), dtype=complex)
CNOT[0][0][0][0] = 1
CNOT[0][1][0][1] = 1
CNOT[1][0][1][1] = 1
CNOT[1][1][1][0] = 1
all_nodes = []
# NodeCollection allows us to store all of the nodes created under this context.
with tn.NodeCollection(all_nodes):
  state_nodes = [
      tn.Node(np.array([1.0 + 0.0j, 0.0 + 0.0j],)) for _ in range(2)
  ]
  qubits = [node[0] for node in state_nodes]
  apply_gate(qubits, H, [0])
  apply_gate(qubits, CNOT, [0, 1])
# We can contract the entire tensornetwork easily with a contractor algorithm
result = tn.contractors.optimal(
    all_nodes, output_edge_order=qubits)
print(result.tensor) # array([0.707+0.j, 0.0+0.j], [0.0+0.j, 0.707+0.j])

[[0.70710678+0.j 0.        +0.j]
 [0.        +0.j 0.70710678+0.j]]


In [73]:
import numpy as np

class Node:
    def __init__(self, tensor, edges):
        self.tensor = tensor
        self.edges = edges
        self.connections = []  # List to store connections with other nodes

    def contract(self, other, self_axis, other_axis):
        result_tensor = np.tensordot(self.tensor, other.tensor, axes=([self_axis], [other_axis]))
        new_edges = [edge for i, edge in enumerate(self.edges) if i != self_axis] + \
                    [edge for i, edge in enumerate(other.edges) if i != other_axis]
        return Node(result_tensor, new_edges)

    def connect(self, other, self_axis, other_axis):
        self.connections.append((other, self_axis, other_axis))
        other.connections.append((self, other_axis, self_axis))

    def print_details(self):
        print("Tensor shape:", self.tensor.shape)
        print("Edges:", self.edges)
        print("Connections:", [(id(node[0]), node[1], node[2]) for node in self.connections])
        print("Tensor data:\n", self.tensor)
        print()

# Example of usage
if __name__ == "__main__":
    a = np.array([1.0 + 0.0j, 0.0 + 0.0j])
    h = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
    # B = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

    node_A = Node(a, ['n'])
    node_B = Node(h, ['n', 'p'])
    node_C = Node(h, ['p'])


    # Connect Node A and Node B over specified axes without contracting
    node_A.connect(node_B, 0, 0)


    # Now contract A and B using the connection info
    connection_info = node_A.connections[0]
    contracted_node = node_A.contract(*connection_info)

    contracted_node.connect(node_C, 0, 0)
    connection_info = contracted_node.connections[0]
    contracted_node2 = contracted_node.contract(*connection_info)

    print("Contracted Node details:")
    contracted_node2.print_details()


Contracted Node details:
Tensor shape: (2,)
Edges: []
Connections: []
Tensor data:
 [1.+0.j 0.+0.j]



In [70]:
contracted_node.

Tensor shape: (2,)
Edges: ['p']
Connections: [(137361448035040, 1, 0)]
Tensor data:
 [0.70710678+0.j 0.70710678+0.j]



In [63]:
contracted_node.print_details()
node_A.print_details()
node_B.print_details()

Tensor shape: (2,)
Edges: ['n', 'p']
Connections: []
Tensor data:
 [0.70710678+0.j 0.70710678+0.j]

Tensor shape: (2,)
Edges: ['m', 'n']
Connections: [(137361447929264, 0, 0)]
Tensor data:
 [1.+0.j 0.+0.j]

Tensor shape: (2, 2)
Edges: ['n', 'p']
Connections: [(137361447927920, 0, 0)]
Tensor data:
 [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

